In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

/home/ylu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import SPACEL
from SPACEL import Scube, Splane

In [2]:
method = "SPACEL"
data_folder = "/home/ylu/project/MOSTA/data/"
results_folder = f"./results/{method}/"
cache_folder = os.path.join(results_folder, 'Scube_outputs')
figures_folder = f"./results/figures/{method}"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(cache_folder).mkdir(parents=True, exist_ok=True)

In [3]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(1, 14)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"E16.5_E2S{i}.MOSTA.h5ad")))

100%|████████████████████████████| 13/13 [00:44<00:00,  3.45s/it]


In [4]:
spatial_key = "spatial"

In [5]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [10]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"
anno_key = "annotation"

In [ ]:
## Perform the SPACEL with provided annotation alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    slice1.obsm['spatial_2D'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial_2D'] = slice2.obsm[spatial_key]
    slice1.obsm['spatial'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial'] = slice2.obsm[spatial_key]
    align_slices = [slice1, slice2]
    time_start = time.time()
    Scube.align(
        align_slices,
        cluster_key=anno_key, 
        n_neighbors = 15, 
        n_threads=10,
        p=2,
        write_loc_path=os.path.join(results_folder, 'Scube_outputs/aligned_coordinates.csv')
    )
    align_slices[0].obsm[key_added] = align_slices[0].obsm['spatial_aligned'].values
    align_slices[1].obsm[key_added] = align_slices[1].obsm['spatial_aligned'].values
    time_end = time.time()
    
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm[spatial_key], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1

    matches = [np.arange(align_slices[0].shape[0]), np.argmin(Distance, axis = 1)]
    
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

In [18]:
import scipy
Distance = scipy.spatial.distance.cdist(align_slices[0].obsm[key_added], align_slices[1].obsm[key_added], metric='euclidean')

In [20]:
np.argmin(Distance, axis = 1)

array([ 2563, 10567,   660, ..., 14692, 16716, 15667])